```
'revenue'：用户的下单购买金额 （目标值）
'gender'  性别 1男 0女 空缺 未知
'age'     年龄
'engaged_last_30'  最近30天有关键操作（社区发帖，买家秀）
'lifecycle',  生命周期  A，B，C 注册6个月内 1年内 2年内
' days_since_last_order '  最近一次下单距今天数  <1说明当天有下单
'previous_order_amount'  以往积累的用户购买金额
'3rd_party_stores'  在非自营店铺购买商品的数量，0说明只在自营店铺购买过
```

- 缺失值处理
  - 性别  可以考虑分成 0 1 未知
  - 其它缺失分类特征也可以考虑用上述办法处理
  - 年龄 可以用均值，中位数 或者数据模型填充
- 可处理成哑变量矩阵的
  - lifecycle
- 单变量分析
  - 数值类型特征describe
- 相关与可视化
  - 分析变量之间的相关性
- 回归模型

#### 相关API

- 导入数据sn_shop  (名字可以任意取)
- sn_shop.isnull().sum()查看缺失情况
- fillna() 缺失值填充
  - sn_shop['age'].fillna(sn_shop.age.mean())
- pd.get_dummies(sn_shop) 可以将类别变量转换成one-hot
- describe()常见统计学指标
- 相关性判断
  - sn_shop.corr()
  - sn_shop.corr()[['revenue']].sort_values('revenue',ascending = False)
  - sns.regplot(x轴变量，y轴变量，数据文件) 进行可视化
- 查看模型结果
  - 自变量系数 model.coef_
  - 截距  model.intercept_
- 模型评估
  - score = model.score(x,y)                      x,y打分
  - predictions = model.predict(x)     计算y预测值
  - error = predictions -y                 计算误差
  - rmse = (error** 2).mean() **.5   rmse
  - mae = abs(error).mean()              mae

In [20]:
import pandas as pd
import numpy as np

In [21]:
ecom = pd.read_csv("./ecom.csv")

In [22]:
ecom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29452 entries, 0 to 29451
Data columns (total 8 columns):
revenue                    29452 non-null float64
gender                     17723 non-null float64
age                        16716 non-null float64
engaged_last_30            17723 non-null float64
lifecycle                  29452 non-null object
 days_since_last_order     29452 non-null float64
previous_order_amount      29452 non-null float64
3rd_party_stores           29452 non-null int64
dtypes: float64(6), int64(1), object(1)
memory usage: 1.8+ MB


In [23]:
ecom.describe()

,revenue,gender,age,engaged_last_30,days_since_last_order,previous_order_amount,3rd_party_stores
count,29452.000000,17723.000000,16716.000000,17723.000000,29452.000000,29452.000000,29452.000000
mean,398.288037,0.950742,60.397404,0.073069,7.711348,2348.904830,2.286059
std,960.251728,0.216412,14.823026,0.260257,6.489289,2379.774213,3.538219
min,0.020000,0.000000,18.000000,0.000000,0.130000,0.000000,0.000000
25%,74.970000,1.000000,50.000000,0.000000,2.190000,773.506250,0.000000
50%,175.980000,1.000000,60.000000,0.000000,5.970000,1655.980000,0.000000
75%,499.990000,1.000000,70.000000,0.000000,11.740000,3096.766500,3.000000
max,103466.100000,1.000000,99.000000,1.000000,23.710000,11597.900000,10.000000


In [24]:
ecom.shape

(29452, 8)

In [25]:
ecom.head()

,revenue,gender,age,engaged_last_30,lifecycle,days_since_last_order,previous_order_amount,3rd_party_stores
0,72.98,1.0,59.0,0.0,B,4.26,2343.870,0
1,200.99,1.0,51.0,0.0,A,0.94,8539.872,0
2,69.98,1.0,79.0,0.0,C,4.29,1687.646,1
3,649.99,NaN,NaN,NaN,C,14.90,3498.846,0
4,83.59,NaN,NaN,NaN,C,21.13,3968.490,4


In [32]:
# 性别缺失值处理
for i in ['man','woman','na']:
    ecom.loc[:,i] = 0
# ecom[['na']] = (ecom[['gender']].isnull()).astype("int")
# ecom[['man']] = (ecom[['gender']].i.astype("int")
ecom[['na']] = ecom[['gender']].replace(to_replace=np.nan,value)

In [33]:
ecom.head()

,revenue,gender,age,engaged_last_30,lifecycle,days_since_last_order,previous_order_amount,3rd_party_stores,man,woman,na
0,72.98,1.0,59.0,0.0,B,4.26,2343.870,0,0,0,0
1,200.99,1.0,51.0,0.0,A,0.94,8539.872,0,0,0,0
2,69.98,1.0,79.0,0.0,C,4.29,1687.646,1,0,0,0
3,649.99,NaN,NaN,NaN,C,14.90,3498.846,0,0,0,1
4,83.59,NaN,NaN,NaN,C,21.13,3968.490,4,0,0,1


In [44]:
ecom[["man"]] = ecom[['gender']].fillna(0).astype("int")
ecom[["woman"]].where(ecom[["man"]]+ ecom[['na']]==0,1,inplace=True)

C:\Users\Administrator\Anaconda3\lib\site-packages\pandas\core\generic.py:7772: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors, try_cast=try_cast)


In [43]:
ecom

,revenue,gender,age,engaged_last_30,lifecycle,days_since_last_order,previous_order_amount,3rd_party_stores,man,woman,na
0,72.98,1.0,59.0,0.0,B,4.26,2343.870,0,1.0,0,0
1,200.99,1.0,51.0,0.0,A,0.94,8539.872,0,1.0,0,0
2,69.98,1.0,79.0,0.0,C,4.29,1687.646,1,1.0,0,0
3,649.99,NaN,NaN,NaN,C,14.90,3498.846,0,0.0,0,1
4,83.59,NaN,NaN,NaN,C,21.13,3968.490,4,0.0,0,1
5,319.99,1.0,80.0,0.0,C,15.13,2811.491,10,1.0,0,0
6,140.00,1.0,60.0,0.0,C,11.45,769.990,4,1.0,0,0
7,51.98,1.0,61.0,0.0,C,3.71,2542.822,1,1.0,0,0
8,587.99,NaN,NaN,NaN,C,22.42,602.980,10,0.0,0,1
9,349.99,NaN,NaN,NaN,C,8.26,298.790,3,0.0,0,1
